In [680]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import average_precision_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import brier_score_loss
from math import sqrt
from sklearn.metrics import jaccard_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import top_k_accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import TimeSeriesSplit
import warnings
warnings.filterwarnings("ignore")

In [681]:
df = pd.read_csv('indicators.csv',parse_dates= True, index_col=0)
df = df.iloc[60: , :]
df

,close,Chaikin_oscillator,CCI,williams_percentage,PSAR,MFI,Keltner_channels,TRIX,Ultimate_oscillator,ad_indicator,...,RSI,ichimoku_cloud,MACD,Stochastic_oscillator,Volume_Weighted_Average_Price,Bollinger_band,Fibonacci_retracement,ADX,return,target
timestamp,,,,,,,,,,,,,,,,,,,,,
2018-03-17 00:00:00+00:00,7883.45,-1,-1,1,-1,-1,1,-1,1,-1,...,1,0,-1,1,-1,0,1,-1.0,-4.067440,-1
2018-03-18 00:00:00+00:00,8215.40,1,-1,0,-1,-1,0,-1,1,1,...,1,0,-1,0,-1,0,1,-1.0,-0.641115,0
2018-03-19 00:00:00+00:00,8623.14,1,0,0,-1,-1,0,-1,1,1,...,1,0,-1,0,-1,0,1,-1.0,4.103592,1
2018-03-20 00:00:00+00:00,8920.71,1,0,0,1,1,0,-1,1,1,...,0,0,-1,0,-1,0,1,-1.0,13.157437,1
2018-03-21 00:00:00+00:00,8911.35,-1,0,0,1,1,0,-1,1,1,...,0,0,-1,0,-1,0,1,-1.0,8.471286,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-31 00:00:00+00:00,28477.29,1,0,0,1,-1,0,-1,1,1,...,0,0,1,0,-1,0,1,1.0,4.408412,1
2023-04-01 00:00:00+00:00,28465.30,-1,0,0,1,-1,0,-1,1,1,...,0,0,-1,0,-1,0,1,1.0,0.385917,0
2023-04-02 00:00:00+00:00,28186.76,-1,0,0,1,-1,0,-1,1,1,...,0,0,-1,0,-1,0,1,1.0,0.532502,0


In [682]:
df.isnull().sum()

close                            0
Chaikin_oscillator               0
CCI                              0
williams_percentage              0
PSAR                             0
MFI                              0
Keltner_channels                 0
TRIX                             0
Ultimate_oscillator              0
ad_indicator                     0
Donchian_channel                 0
SMA                              0
EMA                              0
RSI                              0
ichimoku_cloud                   0
MACD                             0
Stochastic_oscillator            0
Volume_Weighted_Average_Price    0
Bollinger_band                   0
Fibonacci_retracement            0
ADX                              0
return                           0
target                           0
dtype: int64

In [683]:
abs(df.var())

close                            2.763436e+08
Chaikin_oscillator               9.995352e-01
CCI                              4.198299e-01
williams_percentage              4.241862e-01
PSAR                             1.000564e+00
MFI                              1.000352e+00
Keltner_channels                 2.861751e-01
TRIX                             1.000543e+00
Ultimate_oscillator              0.000000e+00
ad_indicator                     9.521282e-01
Donchian_channel                 9.394779e-02
SMA                              1.000146e+00
EMA                              9.977662e-01
RSI                              2.984206e-01
ichimoku_cloud                   2.765425e-01
MACD                             9.969854e-01
Stochastic_oscillator            4.241862e-01
Volume_Weighted_Average_Price    9.998114e-01
Bollinger_band                   1.125845e-01
Fibonacci_retracement            4.933664e-01
ADX                              1.000571e+00
return                           4

In [684]:
X = df.drop(['return','target', 'Ultimate_oscillator'], axis=1)
X = X.astype(int)
X.shape

(1736, 20)

In [685]:
y = df.target
y.value_counts()

 1    604
 0    581
-1    551
Name: target, dtype: int64

In [686]:
window = 150
n_splits = (df.shape[0] - window) // window
#tscv = TimeSeriesSplit(n_splits=n_splits)
cv = KFold(n_splits=n_splits, random_state=0, shuffle=True)
classifier_pipeline = make_pipeline(KNeighborsClassifier(n_neighbors=10))

In [687]:
feature_by_correlation = df.drop(['close','return'], axis=1)

In [688]:
abs(feature_by_correlation.corr()["target"])

Chaikin_oscillator               0.398288
CCI                              0.567032
williams_percentage              0.592590
PSAR                             0.519561
MFI                              0.378691
Keltner_channels                 0.582121
TRIX                             0.326994
Ultimate_oscillator                   NaN
ad_indicator                     0.399423
Donchian_channel                 0.209946
SMA                              0.062133
EMA                              0.094696
RSI                              0.371016
ichimoku_cloud                   0.114128
MACD                             0.309970
Stochastic_oscillator            0.592590
Volume_Weighted_Average_Price    0.065371
Bollinger_band                   0.398106
Fibonacci_retracement            0.022881
ADX                              0.450086
target                           1.000000
Name: target, dtype: float64

In [689]:
abs(feature_by_correlation.corr()["target"][abs(feature_by_correlation.corr()["target"])>0.5].drop('target')).index.tolist()

['CCI',
 'williams_percentage',
 'PSAR',
 'Keltner_channels',
 'Stochastic_oscillator']

In [700]:
vals = [0.1,0.2,0.3,0.4,0.5]
evaluation_metrics = []
for val in vals:
    features = abs(feature_by_correlation.corr()["target"][abs(feature_by_correlation.corr()["target"])>val].drop('target')).index.tolist()
    X = feature_by_correlation.drop(columns='target')
    X=X[features]
    #print(features)
    y_pred = cross_val_predict(classifier_pipeline, X, y, cv=cv)
    y_pred = np.round(y_pred).astype(int)
    evaluation_metrics.append({
        'Accuracy': round(accuracy_score(y,y_pred),2),
        'Precision': round(sqrt(precision_score(y,y_pred, average='weighted')),2),
        'Recall': round(recall_score(y,y_pred, average='weighted'),2),
        'F1 Score': round(f1_score(y,y_pred, average='weighted'),2),
        'Features': features,
        'Total Features': len(features),
        'Threshold': val 
    })
    #print("Precision: " + str(round(sqrt(precision_score(y,y_pred, average='weighted')),2)))
    #print("Recall: " + str(round(recall_score(y,y_pred, average='weighted'),2)))
    #print("F1 Score: " + str(round(f1_score(y,y_pred, average='weighted'),2)))
    #print("Accuracy: " + str(round(accuracy_score(y,y_pred),2)))
    #print("------------------------------------------")
result = pd.DataFrame(evaluation_metrics).sort_values(['Accuracy'], ignore_index=True, ascending=False)
result

,Accuracy,Precision,Recall,F1 Score,Features,Total Features,Threshold
0,0.63,0.79,0.63,0.62,"[Chaikin_oscillator, CCI, williams_percentage,...",14,0.2
1,0.63,0.79,0.63,0.62,"[Chaikin_oscillator, CCI, williams_percentage,...",13,0.3
2,0.61,0.78,0.61,0.61,"[Chaikin_oscillator, CCI, williams_percentage,...",15,0.1
3,0.54,0.73,0.54,0.53,"[CCI, williams_percentage, PSAR, Keltner_chann...",6,0.4
4,0.54,0.73,0.54,0.53,"[CCI, williams_percentage, PSAR, Keltner_chann...",5,0.5


In [692]:
result.Features[1]

['Chaikin_oscillator',
 'CCI',
 'williams_percentage',
 'PSAR',
 'MFI',
 'Keltner_channels',
 'TRIX',
 'ad_indicator',
 'RSI',
 'MACD',
 'Stochastic_oscillator',
 'Bollinger_band',
 'ADX']

In [701]:
sfs1 = SFS(classifier_pipeline, 
           k_features=1, 
           forward=False, 
           scoring='roc_auc_ovo_weighted',
           cv=cv)

X = feature_by_correlation.drop(columns='target')
sfs1.fit(X,y)

SequentialFeatureSelector(cv=KFold(n_splits=10, random_state=0, shuffle=True),
                          estimator=Pipeline(steps=[('kneighborsclassifier',
                                                     KNeighborsClassifier(n_neighbors=10))]),
                          forward=False, k_features=(1, 1),
                          scoring='roc_auc_ovo_weighted')

In [702]:
lof = (pd.DataFrame(sfs1.subsets_).T).sort_values(by='avg_score', ascending=False).head(1).feature_names.values[0]
feature_list = [item for item in lof]
feature_list

['Chaikin_oscillator',
 'CCI',
 'williams_percentage',
 'PSAR',
 'MFI',
 'Keltner_channels',
 'Fibonacci_retracement']

In [703]:
X = feature_by_correlation.drop(columns='target')[feature_list]
y = feature_by_correlation['target']

y_pred = cross_val_predict(classifier_pipeline, X, y, cv=cv)
y_pred = np.round(y_pred).astype(int)
print("Precision: " + str(round(sqrt(precision_score(y,y_pred, average='weighted')),2)))
print("Recall: " + str(round(recall_score(y,y_pred, average='weighted'),2)))
print("F1 Score: " + str(round(f1_score(y,y_pred, average='weighted'),2)))
print("Accuracy: " + str(round(accuracy_score(y,y_pred),2)))

Precision: 0.78
Recall: 0.62
F1 Score: 0.61
Accuracy: 0.62


In [704]:
feature_by_correlation[feature_list + ['target']].corr()

,Chaikin_oscillator,CCI,williams_percentage,PSAR,MFI,Keltner_channels,Fibonacci_retracement,target
Chaikin_oscillator,1.000000,0.086793,-0.249387,0.111922,0.232197,-0.239590,0.009955,0.398288
CCI,0.086793,1.000000,-0.729149,0.653954,0.263792,-0.746567,-0.042023,0.567032
williams_percentage,-0.249387,-0.729149,1.000000,-0.656857,-0.314969,0.728428,0.068012,-0.592590
PSAR,0.111922,0.653954,-0.656857,1.000000,0.156649,-0.543290,-0.053919,0.519561
MFI,0.232197,0.263792,-0.314969,0.156649,1.000000,-0.348707,0.020110,0.378691
Keltner_channels,-0.239590,-0.746567,0.728428,-0.543290,-0.348707,1.000000,0.063145,-0.582121
Fibonacci_retracement,0.009955,-0.042023,0.068012,-0.053919,0.020110,0.063145,1.000000,-0.022881
target,0.398288,0.567032,-0.592590,0.519561,0.378691,-0.582121,-0.022881,1.000000


In [705]:
set1 = set(feature_list)
set2 = set(result.Features[1])

common_elements = list(set1.intersection(set2))
print(common_elements)
print(len(common_elements))

['PSAR', 'Keltner_channels', 'Chaikin_oscillator', 'MFI', 'williams_percentage', 'CCI']
6


In [706]:
feature_by_correlation[common_elements + ['target']].corr()

,PSAR,Keltner_channels,Chaikin_oscillator,MFI,williams_percentage,CCI,target
PSAR,1.000000,-0.543290,0.111922,0.156649,-0.656857,0.653954,0.519561
Keltner_channels,-0.543290,1.000000,-0.239590,-0.348707,0.728428,-0.746567,-0.582121
Chaikin_oscillator,0.111922,-0.239590,1.000000,0.232197,-0.249387,0.086793,0.398288
MFI,0.156649,-0.348707,0.232197,1.000000,-0.314969,0.263792,0.378691
williams_percentage,-0.656857,0.728428,-0.249387,-0.314969,1.000000,-0.729149,-0.592590
CCI,0.653954,-0.746567,0.086793,0.263792,-0.729149,1.000000,0.567032
target,0.519561,-0.582121,0.398288,0.378691,-0.592590,0.567032,1.000000


In [707]:
X = feature_by_correlation.drop(columns='target')[common_elements]
y = feature_by_correlation['target']

y_pred = cross_val_predict(classifier_pipeline, X, y, cv=cv)
y_pred = np.round(y_pred).astype(int)
print("Precision: " + str(round(sqrt(precision_score(y,y_pred, average='weighted')),2)))
print("Recall: " + str(round(recall_score(y,y_pred, average='weighted'),2)))
print("F1 Score: " + str(round(f1_score(y,y_pred, average='weighted'),2)))
print("Accuracy: " + str(round(accuracy_score(y,y_pred),2)))

Precision: 0.78
Recall: 0.62
F1 Score: 0.61
Accuracy: 0.62
